In [44]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import functools
#np.set_printoptions(threshold=np.inf)

In [3]:
# catalog name
catalog = 'groups_088.hdf5'

In [4]:
# value of the scale factor (for redshift 0.05)
scale_factor = 0.95238

# open the catalogue
f = h5py.File(catalog, 'r')

In [5]:
# read the positions, velocities and masses of the FoF halos
pos_h  = f['Group/GroupPos'][:]/1e3           #positions in Mpc/h
vel_h  = f['Group/GroupVel'][:]/scale_factor  #velocities in km/s
mass_h = f['Group/GroupMass'][:]*1e10         #masses in Msun/h

# read the positions, black hole masses and stellar masses of the subhalos/galaxies
pos_g  = f['Subhalo/SubhaloPos'][:]/1e3         #positions in Mpc/h
BH_g   = f['Subhalo/SubhaloBHMass'][:]*1e10     #black-hole masses in Msun/h
M_star = f['Subhalo/SubhaloMassType'][:,4]*1e10 #stellar masses in Msun/h

# close file
f.close()

In [9]:
# find the galaxies
galaxy_pos = pos_g[M_star > 0]
print(str(M_star.shape[0]) + ' subhalos and ' + str(galaxy_pos.shape[0]) + ' galaxies.')
numGals = galaxy_pos.shape[0]

17267 subhalos and 2446 galaxies.


In [32]:
# graph the galaxies
%matplotlib qt
fig = plt.figure(figsize = (8, 8))
ax = fig.add_subplot(projection='3d')
ax.set_xlim(0, 25)
ax.set_ylim(0, 25)
ax.set_zlim(0, 25)
ax.set_xlabel('X [Mpc/h]')
ax.set_ylabel('Y [Mpc/h]')
ax.set_zlabel('Z [Mpc/h]')
ax.scatter(galaxy_pos[:,0], galaxy_pos[:,1], galaxy_pos[:,2], s = 1.5, alpha = 0.7, color = 'red')
plt.tight_layout()
plt.show()

In [33]:
# enumerate wavevectors
gen_ks = np.zeros((1000, 3));
for nx in range(10):
    for ny in range(10):
        for nz in range(10):
            gen_ks[nx*100+ny*10+nz,:] = (2*np.pi/50)*np.array([nx,ny,nz])

ks_oct1 = np.array(list(filter(lambda k: np.linalg.norm(k) < 2*np.pi/5, gen_ks)))
signs = np.array([
    [ 1,  1,  1],
    [ 1,  1, -1],
    [ 1, -1,  1],
    [ 1, -1, -1],
    [-1,  1,  1],
    [-1,  1, -1],
    [-1, -1,  1],
    [-1, -1, -1],])

ks_space = np.unique((signs[:, None, :] * ks_oct1[None, :, :]).reshape(-1, 3), axis=0)

# graph k-grid
%matplotlib qt
fig = plt.figure(figsize = (8, 8))
ax = fig.add_subplot(projection='3d')
ax.set_xlim(-1.4, 1.4)
ax.set_ylim(-1.4, 1.4)
ax.set_zlim(-1.4, 1.4)
ax.set_xlabel('X [Mpc/h]')
ax.set_ylabel('Y [Mpc/h]')
ax.set_zlabel('Z [Mpc/h]')
ax.scatter(ks_space[:,0], ks_space[:,1], ks_space[:,2], s = 1.5, alpha = 0.7, color = 'red')
plt.tight_layout()
plt.show()
print("Our grid has " + str(ks_space.shape[0]) + " ks.")

Our grid has 4139 ks.


In [55]:
# map reduce Fourier transform
coeffs = np.mean(np.exp(1j * (galaxy_pos @ ks_space.T)), axis=0)
print("We have " + str(coeffs.shape[0]) + " complex Fourier coefficients!")

def rhoField(r):
    modes = np.exp(1j * ks_space @ r.T)
    return np.sum(coeffs.T @ modes).real

# visualize the density field
%matplotlib qt
fig = plt.figure(figsize = (8, 8), facecolor='k')
ax = fig.add_subplot(projection='3d', facecolor='k')
ax.tick_params(colors='w')
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False
ax.set_xlim(0, 25)
ax.set_ylim(0, 25)
ax.set_zlim(0, 25)
ax.set_xlabel('X [Mpc/h]', color='w')
ax.set_ylabel('Y [Mpc/h]', color='w')
ax.set_zlabel('Z [Mpc/h]', color='w')
gen_pts = np.zeros((8000, 3));
rho = np.zeros(8000, dtype=complex);
spaced = np.linspace(0, 25, 20)
for nx in range(20):
    for ny in range(20):
        for nz in range(20):
            gen_pts[nx*400+ny*20+nz,:] = np.array([spaced[nx],spaced[ny],spaced[nz]])
            rho[nx*400+ny*20+nz] = rhoField(np.array([spaced[nx],spaced[ny],spaced[nz]]))
ax.scatter(gen_pts[:,0], gen_pts[:,1], gen_pts[:,2], s = 1.5, alpha = 0.7, c = rho, cmap='plasma')
plt.tight_layout()
plt.show()


We have 4139 complex Fourier coefficients!
